# Check PTP Status

In [ ]:
import os
import json
import traceback
import mflib 
print(f"mflib version  {mflib.__version__} " )
from mflib.mflib import MFLib
# import fabrictestbed_extensions
# from fabrictestbed_extensions.fablib.fablib import fablib
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
#import owl as owl
from mflib import owl as owl

## List slices

In [ ]:
fm = fablib_manager()
fm.list_slices()

## Get slice

In [ ]:
%%time

slice_name = "latency_monitoring_slice"

try:
    slice = fm.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print(slice)

In [ ]:
nodes = slice.get_nodes()

In [ ]:
for node in nodes:
    print("\n************************")
    node.execute("hostname")
    node.execute("ps -ef | grep phc2sys")

## Set up PTP (Precision Time Protocol)

## Install and setup linuxptp package  on nodes
Download the Ansible role to configure and install the LinuxPTP software. For more details regarding the steps performed in the playbook, please refer to the repo at [https://github.com/fabric-testbed/ptp](https://github.com/fabric-testbed/ptp)

In [ ]:
pre_requisites = None

# Set Deployment tool repository details
repo_branch = 'main'
repo_name = 'ptp'
destination_folder = f"""/tmp/{repo_name}-{repo_branch}"""
clone_instructions = f"""
cd /tmp/;rm -rf /tmp/{repo_name}-{repo_branch};git clone --branch {repo_branch} https://github.com/fabric-testbed/{repo_name}.git {destination_folder};
"""

### Setting PTP Install Restrictions (Leave it blank)

* If you do not want all interfaces synchronized to PTP, add the name of interfaces to avoid as shown
* Management interfaces are not considered and are avoided by default
* If you do not want the system clock synchronized to PTP set the 'SYNC_SYSTEM_CLOCK' to False
* If you do not have any restrictions for a node, you can omit that node from the list

Example:
```
NODE_RESTRICTIONS = { 
   'node1' : { 'AVOID_IFACES': ['enp6s0'],'SYNC_SYSTEM_CLOCK': False},
   'node2' : { 'AVOID_IFACES': ['enp6s0','enp7s0']},
}
```

In [ ]:
NODE_RESTRICTIONS = {}

### Restrict Ansible operation based on tags (Leave it blank)

* Possible values are ptp_stop,ptp_start,ptp_install 
* Only one tag is allowed
* If empty then all three are performed in the right sequence
* If NODE_RESTRICTIONS are applied along with the tags, the operations will not be performed on the AVOIDED INTERFACES

Example
```
ansible_tags = 'ptp_stop'
```

In [ ]:
ansible_tags = ''

### Run Ansible playbook on each node that needs PTP set up

In [ ]:
bad_nodes = [slice.get_node(name="node5"),]

In [ ]:

# Instruction to run ansible command from the node
ansible_instructions = f"""
cd {destination_folder}/ansible;ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml"""

#Create execute threads
execute_threads = {}

for node in bad_nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    node_name = node.get_name()
    
    # Create JSON files for extra params that will be provided to ansible
    if node_name in NODE_RESTRICTIONS.keys():    
        extra_ansible_params = f""" --extra-vars @parameters.json""";
        with open('/tmp/'+node_name+'-parameters.json', 'w') as f:
            json.dump(NODE_RESTRICTIONS[node_name], f)
        print (f"Uploading install restrictions for {node_name}")    
        node.upload_file('/tmp/'+node_name+'-parameters.json',destination_folder+'/ansible/parameters.json')
    else:
        extra_ansible_params = ''
    if ansible_tags != '':
        extra_ansible_params = extra_ansible_params + ' --tags '+ansible_tags
        
    print (f"Running the PTP Deployment Ansible Playbook on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}"\
                f"{extra_ansible_params}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()

print (f"Ansible Playbook run on all nodes completed\n")